Pandas is the go-to import for working with excel files (and other comma-separated-value [csv] type files). It makes working with large amounts of data, neatly organized in excel type files, simple.

In [59]:
import pandas as pd
import seaborn as sns

file_name = "DRAC_workshop.xls"              # Filename of the excel sheet

# read in the excel or csv type file
#df = pd.read_excel(file_name)
df = sns.load_dataset("titanic")

# if we want to the top X lines, where in this case X = 10 (Default is 5)
print(df.head(10))         

# Similarly we can see what is at the end of the file by using
print(df.tail())
# We can take our dataframe and make an excel file with the following
df.to_excel("titanic.xlsx", index=False)



   survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0         0       3    male  22.0      1      0   7.2500        S   Third   
1         1       1  female  38.0      1      0  71.2833        C   First   
2         1       3  female  26.0      0      0   7.9250        S   Third   
3         1       1  female  35.0      1      0  53.1000        S   First   
4         0       3    male  35.0      0      0   8.0500        S   Third   
5         0       3    male   NaN      0      0   8.4583        Q   Third   
6         0       1    male  54.0      0      0  51.8625        S   First   
7         0       3    male   2.0      3      1  21.0750        S   Third   
8         1       3  female  27.0      0      2  11.1333        S   Third   
9         1       2  female  14.0      1      0  30.0708        C  Second   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes

If we were using a comma separated value (CSV) file to store data we would instead use

In [ ]:
df.read_csv(file_name)

If the read_excel did not work, it might be the case that we need to install openpyxl using

In [ ]:
pip install openpyxl

If we have an excel document with many different sheets in it, we can open a specific sheet by using the sheet_name= keyword.

In [ ]:
df = pd.read_excel("workshop_attendance.xlsx", sheet_name="INSERT REAL SHEET NAME HERE")

If we want to see all of the possible colum headers in the excel file we can use the '.keys()' function

In [ ]:
print(df.keys())

We can see all available sheets in our excel file by using df.keys(), so long as there are multiple sheets present. If there is only 1 sheet, then df.keys() will output the column names. We could also output column names using df.columns()

In [ ]:
print(df.keys())

If our excel file had many sheets, we could open it into a variable 'dfs' or 'sheets' or whatever name we wanted, and then exctract sheets into their own dataframe

In [ ]:
dfs = pd.read_excel("NAME OF EXCEL WITH MULTIPLE SHEETS")
df1 = dfs["NAME OF SHEET 1"]
df2 = dfs["NAME OF SHEET 2"]
dfN = dfs["NAME OF SHEET N"]

When we read in our excel file into 'df' or whatever other variable we want to save it as, the columns are keys, so we can access the data in a column the same way as a key in a dictionary is accessed


In [ ]:
# Access the column called 'fare'
fare = df["fare"]

# we can perform operations such as mean, sum and others by adding .mean() or .sum() etc
print(df["fare"].mean())

import numpy as np
print(np.mean(fare))
# this will print out the average fare

# We could also sum up the fare
print(f"The total fare on the titanic was {df['fare'].sum()}")

remove all NaN

In [ ]:
# To drop all NaN values in the whole sheet
df = df.dropna()

# To drop NaN's in specific columns
df = df.dropna(subset=['column_1', 'column_N'])   # REPLACE WITH REAL COLUMN NAMES

Remove columns where all numerical values are below a specified threshold

In [ ]:
df = df[(df.select_dtypes(include=['number']) >= 10).all(axis=1)] # Keep rows where all values in row are greater or equal to 10

Remove all Columns with values smaller than X, where in this case X = 10

In [ ]:
df = df.loc[(df.select_dtypes(include=['number']) >= 10).all()]

We may want to remove columns that have too many missing values

In [ ]:
threshold = 0.8  # 80% threshold (80% must be present)
df = df.dropna(thresh=len(df) * threshold, axis=1)
print(df.keys())

We may want to count number of unique values in a column, for instance how many unique cities were the passengers from

In [ ]:
print(df["embark_town"].value_counts())

We can count occurances in rows (A little ugly with the lambda function)

In [ ]:
df.apply(lambda row: row.value_counts(), axis=1)

# Machine Learning with Scikit-Learn

We will now use machine learning to try to make a model to predict if a person will survive based on 1) the fare they paid, or 2) where they embarked

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Drop rows with missing target ('survived') and drop NaNs in key columns
df = df.dropna(subset=["survived", "embarked", "fare"])

# Select features and target
features = ["fare", "embarked"]
target = "survived"

# One-hot encode categorical column 'embarked' (take strings, make several columns that are only binary)
# i.e. 'embarked' has options of S, Q or C. This removes 'embarked' columns but adds 'embarked_S', 'embarked_Q' etc
df = pd.get_dummies(df, columns=["embarked"], drop_first=True)

# used embarked_S, embarked_Q, embarked_C
features = ["fare", "embarked_S", "embarked_Q",]

# Split data into train and test sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

# Train a Random Forest classifier (default n_estimators is 100)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.69


Now that we have trained our model, we can use it to make predictions.

We just need to supply the feature values for a new candidate.


In [58]:
# Create new passenger. Note all values are lists since Pandas expects columns of values
Gerald = pd.DataFrame({'fare': [30], 'embarked_Q':[0], 'embarked_S':[1]})
Gerald_prediction = model.predict(Gerald)
print(f"The predicted survivability of Gerald is: {Gerald_prediction[0]}")

The predicted survivability of Gerald is: 0
